# Image Patchify + Unpatchify

In [66]:
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from glob import glob
from PIL import Image
import cv2
import os

In [67]:
path = "C:/intern_data/"

In [68]:
large_image_stack = glob(path+'image/Panorama/*.jpg')

In [69]:
print(len(large_image_stack))

1


In [ ]:
# unpatchify에 필요한 image.shape의 크기 
# 필요한 가로 크기 : 448*(69-1) + 512 = 30976 
                    # 따라서 31210-30976 = 234 픽셀 없어져야함
# 필요한 세로 크기 : 448*(2-1) +512 = 960

In [57]:
## unpatchify - image crop  
# 방법 1 : 처음부터 이미지 크기 자르기 
for img in range(len(large_image_stack)):
    image_crop = Image.open(large_image_stack[img])
    croppedImage= image_crop.crop((0,0,30976,960))  # crop(width_start,height_start,width,height)
    croppedImage.save(large_image_stack[img])

In [80]:
for img in range(len(large_image_stack)):
    ## image patchify
    image = cv2.imread(large_image_stack[img],1)
    patches = patchify(image, (512,512,3), step = 448)  #Step= 448 means 64 (512-448) overlap 
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):   
            single_patch_img = patches[i,j,0] # 데이터가 5개면 shape이 10번 찍힘           
            cv2.imwrite(path+"debug_image/" + "image_" + str(img) + "_" + str(i)+str(j)+ ".jpg", single_patch_img)
    
    ## unpatchify 
    # 방법 2: unpatchify 이미지.shape을 임의로 바꿔주기 
    reconstructed_image = unpatchify(patches, (960,30976,3)) # image.shape를 잘린 것처럼 바꿔줘야함. 
    name = large_image_stack[img].split('\\')[1]
    cv2.imwrite(path + "unpatchify_image/"+"unpatch_"+name,reconstructed_image)

In [77]:
patches.shape

(2, 69, 1, 512, 512, 3)

In [6]:
# mask file index create
large_image_stack_mask = []

for name in large_image_stack:
    word = path+"mask\\" + name[30:-4] + "_mask"+".jpg"
    large_image_stack_mask.append(word)

In [7]:
print(len(large_image_stack_mask))

1


In [8]:
## mask patchify
for img in range(len(large_image_stack_mask)):    
    image = cv2.imread(large_image_stack_mask[img],1)   
    patches = patchify(image, (512,512,3), step = 448)  #Step= 448 means 64 overlap
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):                        
            single_patch_img = patches[i,j,0]     
            cv2.imwrite(path+"debug_mask/" + "image_" + str(img) + "_" + str(i)+str(j)+ ".jpg", single_patch_img)

### patchfiy_image function

In [ ]:
def patchfiy_image(large_image_stack,path):
    
    for img in range(len(large_image_stack)):
        image = cv2.imread(large_image_stack[img],1)
        patches = patchify(image, (512,512,3), step = 448)  #Step= 448 means 64 overlap
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):   
                single_patch_img = patches[i,j,0] # 데이터가 5개면 shape이 10번 찍힘           
                cv2.imwrite(path+"debug_image/" + "image_" + str(img) + "_" + str(i)+str(j)+ ".jpg", single_patch_img)
                
    # mask file index create
    large_image_stack_mask = []

    for name in large_image_stack:
        word = path+"mask\\" + name[30:-4] + "_mask"+".jpg"
        large_image_stack_mask.append(word)
        
    ## mask patchify
    for img in range(len(large_image_stack_mask)):    
        image = cv2.imread(large_image_stack_mask[img],1)   
        patches = patchify(image, (512,512,3), step = 448)  #Step= 448 means 64 overlap
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):                        
                single_patch_img = patches[i,j,0]     
                cv2.imwrite(path+"debug_mask/" + "image_" + str(img) + "_" + str(i)+str(j)+ ".jpg", single_patch_img)


## Image Pick

In [9]:
from glob import glob
import cv2
import shutil
import os 

In [10]:
path = "C:/intern_data/"

In [11]:
mask_files = glob(path+'debug_mask/*.jpg')
image_files = glob(path+'debug_image/*.jpg')

In [12]:
## mask pick

mask_list = [] # mask name collect

for file in range(len(mask_files)):
  
    mask = cv2.imread(mask_files[file])
    
    if mask.sum() != 0 :
        mask_name = mask_files[file]     
        shutil.copy(mask_name,path+'yes_mask/')
        mask_list.append(mask_name.split('/')[2].split('\\')[1])

In [13]:
## image pick
for image_name in image_files: # 순서 다를 수 있으므로 안에 이름들 가져오기 
    splited_img_name = image_name.split('/')[2].split('\\')[1]
          
    if splited_img_name in mask_list:
        shutil.copy(image_name,path+'yes_image/')
    

## Image Unpatchify